In [1]:
#Use this file to create basic operations for yank & Unyank
# Last updated on 2020-04-13 by yingyy@, original code from liuyuez@ in 2020-07

In [1]:
import json
import time
import os
import shutil
import re
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [52]:
class chrome_driver(object):
    def __init__(self):
        chrome_options = Options()
        chrome_options.add_argument('disable-infobars')
        chrome_path = os.environ['USERPROFILE'] + r'\AppData\Local\Google\Chrome\User Data\Default'
        chrome_options.add_argument("user-data-dir=" + chrome_path)
        self.driver = webdriver.Chrome(options=chrome_options)
        self.wait = WebDriverWait(self.driver, 300)
    def get_url(self,url):
        self.driver.get(url)
        while 'midway' in self.driver.current_url:
            time.sleep(20)
    def wait_click(self,element_id):
        self.wait.until(EC.element_to_be_clickable((By.ID, element_id)))
    def clickable_click(self,xpath):
        self.wait.until(EC.element_to_be_clickable((By.XPATH, xpath))).click()
    def clickable_click_by_id(self,element_id):
        self.wait.until(EC.element_to_be_clickable((By.ID, element_id))).click()
    def wait_until_located(self,xpath):
        item = self.wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        return item
    def wait_until_all_located(self,xpath):
        item_list = self.wait.until(EC.presence_of_all_elements_located((By.XPATH, xpath)))
        return item_list

In [53]:
# Function to read excel file and return ASIN list by Source-Target pair
def read_excel(file, action):
    df_original = pd.read_excel(file,sheet_name = 'Sheet1')
    df_original = df_original[['ASIN','Source','Target']]
    list_arc_pair = df_original[['Source','Target']].drop_duplicates()
    upload_file_name_list = []
    for index, row in list_arc_pair.iterrows():
        upload_file_name = row[0].strip()+'_'+row[1].strip()+'_'+action+'.txt'
        df_filtered = df_original[(df_original['Source'] == row[0]) & (df_original['Target'] == row[1])]['ASIN']
        df_filtered.ASIN=df_filtered.ASIN.apply(lambda x: str(x).zfill(10))
        generate_upload_file(upload_file_name,df_filtered)
        upload_file_name_list.append(upload_file_name)
    return upload_file_name_list

In [54]:
# Function to generate upload file(s) for yank/unyank per ARC
def generate_upload_file(filename, asin_list):
    with open(filename,'w') as f:
        for asin in asin_list:
            f.writelines(str(asin)+'\n')

In [55]:
# Function to select source/destination marketplace from the dropdown on AGS webtool page
def click_option(chrome, driver, element_id, xpath, option_text):
    chrome.clickable_click_by_id(element_id)
    time.sleep(2)
    options = driver.find_element_by_xpath(xpath).find_elements_by_tag_name('li')
    for option in options:
        if option.text == option_text:
            option.click()
            break
    time.sleep(2)
    
    
# Function to upload a file to yank/unyank
def upload_file(chrome, driver, agsurl, sourcemp, destmp, filename, reason):
    chrome.get_url(agsurl)
    click_option(chrome, driver, 'sourceMarketplace', '//*[@id="a-popover-1"]/div/div/ul', sourcemp)
    click_option(chrome, driver, 'destinationMarketplace', '//*[@id="a-popover-2"]/div/div/ul', destmp)
    driver.find_element_by_id('file').send_keys(filename)
    driver.find_element_by_id('reason').send_keys(reason)
    chrome.clickable_click('//*[@id="a-autoid-1"]/span/input')
    time.sleep(3)
    try:
        tracking = driver.find_element_by_xpath('//*[@id="a-page"]/div[2]/div/div[2]/div/p/a').get_attribute('href')
    except NoSuchElementException:
        tracking = ''
    return tracking

In [24]:
# Function to copy folders containing upload files to WorkDocs 
def copy_folder(source_path, target_path):
    for root, dirs, files in os.walk(source_path):
        for f in files:
            source_file = os.path.join(source_path, f)
            target_folder = os.path.join(target_path,os.path.basename(source_path))
            mkdir(target_folder)
            shutil.copy(source_file, target_folder)
        for d in dirs:
            source_folder = os.path.join(source_path, d)
            target_folder = os.path.join(target_path, d)
            mkdir(target_folder)
            for c_root, c_dirs, c_files in os.walk(source_folder):
                for file in c_files:
                    source_file = os.path.join(source_folder, file)
                    shutil.copy(source_file, target_folder)

# Function to create folder for SIM/upload files management
def mkdir(path):
    path = path.strip()
    path = path.rstrip("\\")
    isExists = os.path.exists(path)
    if not isExists:
        os.makedirs(path)

In [58]:
# Functions to process all the SIMs that meet criteria in the SIM folder
def process_request(filename, taskinfo, requester):
# Assign Variables: Source & Target marketplaces, local_directory and variables
    s_arcs = ['US','GB','DE','JP']
    t_arcs = ['CN','TR','SG']
    mkdir(r'C:\User\yank-ops')
    rool = re.compile('\s+')
    upload_file_list = []
    os.chdir(r'C:\User\yank-ops')
    action = taskinfo.strip().split('_')[1].strip().lower()
    if action == 'unyank':
        action = 'unblock'
# Create a folder named by current date under the same path as this script
    taskinfo_time = taskinfo +'_'+ datetime.now().strftime('%Y%m%d%H%M%S')
    source_path = os.path.join(os.getcwd(), taskinfo_time)
    mkdir(source_path)
    os.chdir(source_path)
# read input excel file and split to .txt files by arc
    upload_file_list = read_excel(filename, action)
# collect reason
    reason = requester.strip().lower()+'_'+taskinfo.split('_')[0]+'_'+taskinfo.split('_',2)[2]
# open chrome
    chrome = chrome_driver()
    driver = chrome.driver
# yank/unyank operations
    tracking_list = []
    for file in upload_file_list:
        agsurl = 'https://ags.amazon.com/' + action
        source = file.split('_')[0].strip().upper()
        target = file.split('_')[1].strip().upper()
        if target == 'UK':
            target = 'GB'
        filepath = os.path.join(os.getcwd(), file)
        tracking = upload_file(chrome, driver, agsurl, source, target, filepath, reason)
        if tracking == '':
            tracking_list.append(file+" failed")
        else: 
            tracking_list.append(tracking)
    driver.close()
# Generate output tracking file
    generate_upload_file('tracking_result.txt',tracking_list)
# WorkDocs path to which upload files will be copied
# Note that WorkDocs Drive must be installed
    target_path = 'W:\\My Documents\\Yank&Unyank - RPA'
    if not os.path.exists(target_path):
        target_path = target_path.replace('My Documents','Shared With Me')
    copy_folder(source_path, target_path)

In [28]:
#target_path = 'W:\\My Documents\\Yank&Unyank - RPA'
#source_path = source_path = os.path.join('C:\\User\\yank-ops', "202104140150_yank_CX_Impact_Deliver_Issue-DS_20210414113110")

In [59]:
# filename = "C:/Users/yingyy/Desktop/Template.xlsx"
# taskinfo = '202104130619_yank_order fulfillment'
# requester = 'yingyy'

In [61]:
#process_request(filename,taskinfo,requester)